In [ ]:
# https://www.analyticsvidhya.com/blog/2018/11/implementation-faster-r-cnn-python-object-detection/
# https://machinelearningmastery.com/how-to-train-an-object-detection-model-with-keras/

In [1]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
import os
import xml.etree.ElementTree as ET
import pandas as pd


#    for file in files:
#        tree = ET.parse(file)
#        root = tree.getroot()
        


In [3]:
import glob

# files = glob.glob('C:\Users\mysky\Documents\Decoded\*.xml')

files = glob.glob('/home/scar3crow/Downloads/BCCD_Dataset-master/BCCD/Annotations/*.xml')

In [118]:
len(files)

364

In [43]:
print(type(files))
print(files[1])

<class 'list'>
/home/scar3crow/Downloads/BCCD_Dataset-master/BCCD/Annotations/BloodImage_00057.xml


In [44]:
tree = ET.parse(files[1])
root = tree.getroot()

In [45]:
# extract each bounding box
for box in root.findall('.//bndbox'):
    xmin = int(box.find('xmin').text)
    ymin = int(box.find('ymin').text)
    xmax = int(box.find('xmax').text)
    ymax = int(box.find('ymax').text)
    coors = [xmin, ymin, xmax, ymax]

In [46]:
coors

[410, 290, 503, 390]

In [38]:
root.tag

'annotation'

In [47]:
root[1].text

'BloodImage_00057.jpg'

In [48]:
print(root[1].text)

BloodImage_00057.jpg


In [51]:
# function to extract bounding boxes from an annotation file
def extract_boxes(filename):
    # load and parse the file
    tree = ET.parse(filename)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    boxes = list()
    for box in root.findall('.//bndbox'):
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        coors = [xmin, ymin, xmax, ymax]
        boxes.append(coors)
    # extract image dimensions
    i_name = root[1].text    # since image name is in position [1] of .xml file - see .xml file
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    return boxes, width, height, i_name

In [158]:
# function to extract bounding boxes from an annotation file
def extract_boxes(filename):
    
    blood_frame = pd.DataFrame()
    # load and parse the file
    tree = ET.parse(filename)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    boxes = list()
    i_name = root[1].text    # since image name is in position [1] of .xml file - see .xml file
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    
    
    for box in root.findall('.//object'):
        
        c_name = box.find('name').text
        
        xmin = int(box.find('.//bndbox/xmin').text)
        ymin = int(box.find('.//bndbox/ymin').text)
        xmax = int(box.find('.//bndbox/xmax').text)
        ymax = int(box.find('.//bndbox/ymax').text)
        coors = [i_name, c_name, xmin, xmax, ymin, ymax, width, height]
        boxes.append(coors)
    blood_frame = pd.DataFrame(boxes)
    return blood_frame

In [159]:
# extract details form annotation file
train = extract_boxes(files[1])
# summarize extracted details
train

,0,1,2,3,4,5,6,7
0,BloodImage_00057.jpg,RBC,488,596,252,357,640,480
1,BloodImage_00057.jpg,RBC,393,487,127,214,640,480
2,BloodImage_00057.jpg,RBC,460,565,64,172,640,480
3,BloodImage_00057.jpg,RBC,5,110,319,427,640,480
4,BloodImage_00057.jpg,RBC,221,322,315,406,640,480
5,BloodImage_00057.jpg,RBC,267,364,17,117,640,480
6,BloodImage_00057.jpg,WBC,165,317,71,215,640,480
7,BloodImage_00057.jpg,Platelets,549,585,405,444,640,480
8,BloodImage_00057.jpg,Platelets,513,549,401,440,640,480
9,BloodImage_00057.jpg,RBC,322,415,291,391,640,480


In [160]:
col_names = ['image_names', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax', 'width', 'height']

b_frame = pd.DataFrame()

for i in range(len(files)):
#    b_frame.append(extract_boxes(files[i]))
    b_frame = pd.concat([b_frame, extract_boxes(files[i])])
    
b_frame.columns = col_names

b_frame = b_frame.reset_index(drop = True)
    
#blood_frame = pd.DataFrame(blood_data, columns = col_names)

In [161]:
b_frame

,image_names,cell_type,xmin,xmax,ymin,ymax,width,height
0,BloodImage_00111.jpg,RBC,458,571,198,296,640,480
1,BloodImage_00111.jpg,RBC,405,518,240,338,640,480
2,BloodImage_00111.jpg,RBC,365,478,40,138,640,480
3,BloodImage_00111.jpg,RBC,512,630,75,188,640,480
4,BloodImage_00111.jpg,RBC,468,570,71,175,640,480
5,BloodImage_00111.jpg,RBC,175,288,169,267,640,480
6,BloodImage_00111.jpg,RBC,278,391,358,456,640,480
7,BloodImage_00111.jpg,RBC,527,640,306,404,640,480
8,BloodImage_00111.jpg,RBC,229,342,31,129,640,480
9,BloodImage_00111.jpg,RBC,23,136,44,142,640,480


In [162]:
len(b_frame)

4888